In [1]:
from model import *
from data import *

Using TensorFlow backend.


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [2]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

/Users/adib/Downloads/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/5
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
2000/2000 [==============================] - 9154s 5s/step - loss: 0.2300 - acc: 0.9011

Epoch 00001: loss improved from inf to 0.23003, saving model to unet_membrane.hdf5
Epoch 2/5
2000/2000 [==============================] - 8589s 4s/step - loss: 0.1265 - acc: 0.9445

Epoch 00002: loss improved from 0.23003 to 0.12650, saving model to unet_membrane.hdf5
Epoch 3/5
2000/2000 [==============================] - 8561s 4s/step - loss: 0.0984 - acc: 0.9569

Epoch 00003: loss improved from 0.12650 to 0.09842, saving model to unet_membrane.hdf5
Epoch 4/5
2000/2000 [==============================] - 8632s 4s/step - loss: 0.0832 - acc: 0.9636

Epoch 00004: loss improved from 0.09842 to 0.08325, saving model to unet_membrane.hdf5
Epoch 5/5
2000/2000 [==============================] - 10486s 5s/step - loss: 0.0744 - acc: 0.9675

Epoch 00005: loss improved from 0.08325 to 0.07443, saving model to unet_membran

### Train with npy file

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [3]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)

/Users/adib/.pyenv/versions/3.5.6/envs/ml-3.5/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/adib/.pyenv/versions/3.5.6/envs/ml-3.5/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


30/30 [==============================] - 20s 679ms/step


/Users/adib/.pyenv/versions/3.5.6/envs/ml-3.5/lib/python3.5/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
